In [12]:
import numpy as np
from sklearn.svm import SVC
from skimage.feature import hog
from skimage import data, color, exposure
from skimage.color import rgb2gray
import matplotlib.pyplot as plt

#Xtr = np.genfromtxt ('/home/moriarty/Desktop/DK/ENS/kernel_methods_MVAproject-master/Xtr.csv', delimiter=",")
#Xtr=Xtr[:,:-1]
Xte = np.genfromtxt ('/home/moriarty/Desktop/DK/ENS/kernel_methods_MVAproject-master/Xte.csv', delimiter=",")
Xte=Xte[:,:-1]
#Xtrain = np.load('data/Xtrain.npy')
#Ytrain = np.load('data/Ytrain.npy')

In [15]:
#Xtrain = Xtr.reshape(5000,32,32,3,order='F')
#for i in [109,189,195]:
#    lig = Xtrain[i,:]
#    plt.imshow(lig)
#    plt.show()
#    image=rgb2gray(Xtrain[i,:])
#    plt.imshow(image)
#    plt.show()
print Xte.shape
Xtest = Xte.reshape(2000,32,32,3,order='F')

(2000, 3072)


In [17]:
#L=[]
#for i in xrange(Xtrain.shape[0]):
#    L.append(rgb2gray(Xtrain[i,:]))
#XtrainGrey = np.asarray(L)
#print XtrainGrey.shape
#for i in [109,189,195]:
#    lig = XtrainGrey[i,:]
#    plt.imshow(lig)
#    plt.show()
L=[]
for i in xrange(Xtest.shape[0]):
    L.append(rgb2gray(Xtest[i,:]))
XtestGrey = np.asarray(L)
print XtestGrey.shape

(2000, 32, 32)


In [20]:
from sklearn.decomposition import PCA
import numpy
from tqdm import tqdm

from fisher_vector import FisherVector
from hog_feature_extractor import HOGFeatureExtractor

class FisherFeatureExtractor:
    """
    nbins: number of bins that will be used
    unsigned: if True the sign of the angle is not considered
    """
    def __init__(self, local_feature_extractor='hog', nclasses=10, nbins=9, unsigned=True):
        self.local_feature_extractor = local_feature_extractor
        self.nclasses = nclasses
        self.nbins = nbins
        self.unsigned = unsigned
        
    def predict(self, X):
        assert X.ndim == 4
        print("Extracting Fisher features")
        n = X.shape[0]
        ret = []
        
        local_features = None
        if self.local_feature_extractor == 'hog':
            hog = HOGFeatureExtractor(nbins=self.nbins, unsigned=self.unsigned)
            local_features = hog.predict(X, unflatten=False)
            print local_features.shape
            pca = PCA(n_components=66)# adjust yourself
            pca.fit(local_features)
            local_features = pca.transform(local_features)
            print local_features.shape
            local_features_fisher = local_features.reshape(5000,3,22)
        else:
            raise Exception("Unknown local feature extractor")
        
        fisher_vector = FisherVector(self.nclasses)

        for i in tqdm(range(n)):
            ret.append(fisher_vector.predict(local_features_fisher[i,:,:]))
            continue

        return numpy.array(ret)
        #pca = PCA(n_components=64)# adjust yourself
        #pca.fit(ret)
        #ret_pca = pca.transform(ret)
        #return numpy.array(ret_pca)

In [21]:
FFE = FisherFeatureExtractor(local_feature_extractor='hog', nclasses=10, nbins=9, unsigned=True)
XtrainHog = FFE.predict(Xtrain)

  0%|          | 3/5000 [00:00<03:44, 22.29it/s]

Extracting Fisher features
Extracting HOG features


100%|██████████| 5000/5000 [03:25<00:00, 24.28it/s]


(5000, 972)


  0%|          | 0/5000 [00:00<?, ?it/s]

(5000, 66)


100%|██████████| 5000/5000 [20:24<00:00,  3.53it/s]


In [5]:
print local_features.shape

NameError: name 'local_features' is not defined

In [21]:
#L=[]
#for i in xrange(XtrainGrey.shape[0]):
#    L.append(hog(XtrainGrey[i,:], orientations=9,pixels_per_cell=(6, 6)))
##, pixels_per_cell=(2, 2),cells_per_block=(4, 4))
#XtrainHOG = np.asarray(L)
L=[]
for i in xrange(XtestGrey.shape[0]):
    L.append(hog(XtestGrey[i,:], orientations=9,pixels_per_cell=(6, 6)))
#, pixels_per_cell=(2, 2),cells_per_block=(4, 4))
XtestHOG = np.asarray(L)

In [9]:
#numpy.save('data/XtrainHog_pca_fisher', XtrainHog)
#print XtrainHog.shape
Ytr = np.genfromtxt ('/home/moriarty/Desktop/DK/ENS/kernel_methods_MVAproject-master/Ytr.csv', delimiter=",",skip_header=1)
Ytr=Ytr[:,1]
Ytr=np.array([Ytr])
print Ytr.T.shape

(5000, 1)


In [18]:
print Ytr.shape
print XtrainHOG.shape
data=np.concatenate((XtrainHOG,Ytr.T),axis=1)
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]
Xtrain=train[:,:-1]
Ytrain=train[:,-1]
Xtest=test[:,:-1]
Ytest=test[:,-1]

C_values=[0.1,1,10,20,50,100,1000,5000,10000,25000,500000]
acc_train_list=[]
acc_test_list=[]
for i in C_values:
    clf = SVC(C=i,kernel='linear',verbose=False,gamma='auto')
    clf.fit(Xtrain, Ytrain)
    y_hat=clf.predict(Xtest)
    correct = np.sum(y_hat == Ytest)
    acc=correct/float(len(y_hat))
    acc_test_list.append(acc)

    y_hat_train=clf.predict(Xtrain)
    #y_kaggle = clf.predict()
    correct_train = np.sum(y_hat_train == Ytrain)
    acc_train=correct_train/float(len(y_hat_train))
    acc_train_list.append(acc_train)
    #print 'gamma', j
    print 'C', i
    print 'accuracy on test' , acc
    print 'accuracy on train' , acc_train

(1, 5000)
(5000, 729)
C 0.1
accuracy on test 0.132505175983
accuracy on train 0.139811601388
C 1
accuracy on test 0.420289855072
accuracy on train 0.495290034705
C 10
accuracy on test 0.418219461698
accuracy on train 0.604362915221
C 20
accuracy on test 0.41718426501
accuracy on train 0.64154685176
C 50
accuracy on test 0.413043478261
accuracy on train 0.691373326723
C 100
accuracy on test 0.404761904762
accuracy on train 0.73599405057
C 1000
accuracy on test 0.38198757764
accuracy on train 0.906544372831
C 5000
accuracy on test 0.368530020704
accuracy on train 0.980168567179
C 10000
accuracy on test 0.368530020704
accuracy on train 0.993802677243
C 25000
accuracy on test 0.365424430642
accuracy on train 0.99975210709
C 500000
accuracy on test 0.364389233954
accuracy on train 1.0


In [25]:
C_values=[1,5]
for i in C_values:
    clf = SVC(C=i,kernel='linear',verbose=False,gamma='auto')
    clf.fit(Xtrain, Ytrain)
    y_kaggle = clf.predict(XtestHOG)
    print i
    np.savetxt("y"+str(i)+".csv", y_kaggle, delimiter=",")

1
5
